<a href="https://colab.research.google.com/github/psychopatz/Colab-Torrent-Downloader-RELOADED/blob/main/Colab_Torrent_Downloader_Reloaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://bit.ly/3xlijDv" target="_blank">
<img src="https://icons.iconarchive.com/icons/neiio/prime-dock-2/256/uTorrent-icon.png" height="100px" align="left"></a>

# <font size=6>Torrent Download <font color=#e53c1d>RE</font><font color=#16bf2d>LOA</font><font color=#158ff5>DEAD</font></font>



# A feature-rich **Torrent Download manager** powered by Google Collab.
**Features:**

<ul type- "disc">

<li>Built-in Torrent Search engine for fast secure search.</li>
<li>History and resume downloads support.</li>
<li>VPN-like features both downloading and searching that prevents DMCA Notice Since it technically uses Google's internet instead of yours.</li>
</ul>

Visit https://bit.ly/3xlijDv for more info.




In [ ]:
#@title Initialize Downloader

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
!pip install bencodepy
!pip install tpblite



import os
import sys
import time
import shutil
import base64
import hashlib
import bencodepy
import libtorrent as lt
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from google.colab import output
from google.colab import files
from tpblite import TPB, CATEGORIES, ORDERS

time.sleep(1)
clear_output()
print('Importing Done..Now Authenticating GDrive ,')

#Drive Authentication
from google.colab import drive
drive.mount("/content/drive")


#Environmental Variables
TorrentHistory = '/content/drive/My Drive/Torrent/TorrentHistory/'
SaveDirectory = "/content/drive/My Drive/Torrent" #@param {type:"string"}
completed_path = TorrentHistory + 'Completed/'

params = {"save_path": SaveDirectory}

for path in [TorrentHistory, SaveDirectory, completed_path]:
 if not os.path.exists(path):
	  os.makedirs(path)

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

color = {
   "PURPLE":'\033[95m',
   "CYAN":'\033[96m',
   "DARKCYAN":'\033[36m',
   "BLUE":'\033[94m',
   "GREEN":'\033[92m',
   "YELLOW":'\033[93m',
   "RED":'\033[91m',
   "BOLD":'\033[1m',
   "UNDERLINE":'\033[4m',
   "END":'\033[0m'
   
   }

######Functions########
print ('Initialize Required Functions',end='')
# extract magnet from .torrent
def torrent2Magnet(file) :
    metadata = bencodepy.decode_from_file(file)
    subj = metadata[b'info']
    hashcontents = bencodepy.encode(subj)
    digest = hashlib.sha1(hashcontents).digest()
    b32hash = base64.b32encode(digest).decode()
    return 'magnet:?'\
             + 'xt=urn:btih:' + b32hash\
             + '&dn=' + metadata[b'info'][b'name'].decode()\
             + '&tr=' + metadata[b'announce'].decode()\
             + '&xl=' + str(metadata[b'info'][b'piece length'])


#Verify the magnet and download its metadata
def magnet2Torrent(magnet_link,TorrentHistory):
    try:
        params = lt.parse_magnet_uri(magnet_link)
    except:
        print("%sError%s: Only valid Magnet link is accepted!" % (color["RED"],color["END"]))
        return False

    session = lt.session()
    handle = session.add_torrent(params)
    timer = 0
    print("Downloading Metadata", end='')
    while not handle.has_metadata(): 
        timer = timer + 1
        time.sleep(0.1)
        if timer == 100:
            print(".",end='')
        if timer == 250:
            print(".",end='')
        if timer == 500:
            print(".",end='')
        if timer == 750:
            print(".",end='')
        if timer == 850:
            print("It's slow due to %sLow Seeders%s" % (color["RED"],color["END"]))

        if timer >= 1000: #1 min estimation
            print("%sTimeout Error%s, can't download torrent metadata. Try next time or use other magnet." % (color["RED"],color["END"]))
            return False 
        

# Create .torrent and save to the history
    torrent_info = handle.get_torrent_info()
    torrent_file = lt.create_torrent(torrent_info)
    torrent_path = os.path.join(TorrentHistory, torrent_info.name() + ".torrent")
    
    with open(torrent_path, "wb") as f:
        f.write(lt.bencode(torrent_file.generate()))
    print ("\n %s is Added" % torrent_info.name())
    return True

#Show Queue
def showQueue():
  if downloads:
    print ("\n%sDownload Queue%s:" % (color["BOLD"],color["END"]))
    for index, download in enumerate(downloads[:]):
        print ((index + 1),".", download.name())

print ('.',end='')

#Scan for .torrents
def showUnfinished():
  paths = os.listdir(TorrentHistory)
  return [path for path in paths if path.endswith('.torrent')]
 
#Transfer .torrent when completed 
def transferTorrent(torrentName):
  try:
    shutil.move(TorrentHistory + torrentName + '.torrent', completed_path + torrentName + '.torrent')
  except:
    return



#Load All .torrent
def resumeTorrent():
    #Scan for .torrents
    paths = showUnfinished()

    #Add .torrent
    for data in paths:
        magnet_link = torrent2Magnet(TorrentHistory + data)
        #print (magnet_link)
        downloads.append(lt.add_magnet_uri(ses, magnet_link, params))



print ('.',end='')
#add Magnet Downloads
def addViaMagnet():
    while True:
        showQueue()
        magnet_link = input("Enter Magnet Link Or Type Exit: ")
        clear_output()
        if magnet_link.lower() == "exit":
            break
        if magnet2Torrent(magnet_link,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))

#add .torrent Downloads
def addViaTorrent():
    while True:
        showQueue()
        source = files.upload()
        filename = next(iter(source))
        magnet_link = torrent2Magnet(filename)
        if magnet2Torrent(magnet_link,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))
        del source
        clear_output()

############Torrent Searcher###########



print ('.')

print('Importing Done..Mounting GDrive Done,')
print('Initializiation Complete,')
time.sleep(1)
print('Now you can run the next cell')



In [ ]:
#@title Use the Program


def bannerTorrentDownloader():
    print("%s     _____                         _    %s______                   %s_                  _           " % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s    |_paT_|                       | |   %s|  _  \                 %s| |                | |          " % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | ___  _ __ _ __ ___ _ __ | |_  %s| | | |_____      ___ __%s| |  ___   __ _  __| | ___ _ __ " % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | |/ _ \| '__| '__/ _ | '_ \| __| %s| | | / _ \ \ /\ / | '_ \| |/ _ \%s / _` |/ _` |/ _ | '__|" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | (_) | |  | | |  __| | | | |_  %s| |/ | (_) \ V  V /| | | | | (_) | (_| | (_| |  __| |   " % (color["GREEN"],color["BLUE"]))
    print("%s      \_/\___/|_|  |_|  \___|_| |_|\__| %s|___/ \___/ \_/\_/ |_| |_|_|\___/ \__,_|\__,_|\___|_|   " % (color["GREEN"],color["BLUE"]))
    print("%s                                                                                                   " % (color["END"]))

###Beta Functions

def bannerTorrentSearcher():
    print("%s     _____ %s                         _   %s  _____                     _       " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s    |_   _|%s                        | |  %s /  ___|                   | |      " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | |%s  ___  _ __ _ __ ___ _ __ | |_ %s \ `--.  ___  __ _ _ __ ___| |__    " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | %s|/ _ \| '__| '__/ _ | '_ \| __|%s  `--. \/ _ \/ _` | '%s__/ __| '_ \ " % (color["PURPLE"],color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | (_) | |  | | |  __| | | | |_ %s /\__/ |  __| (_| | |%s | (__| | | |" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      \_/\___/|_|  |_|  \___|_| |_|\__|%s \____/ \___|\__,_|_|%s  \___|_| |_|" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s                                                                             " % (color["END"]))
    
def printContents(torrents):
    index=0
    for torrent in torrents:
        index = index + 1
        #print("[%d]%s , %sSeeds: %s %-5s%s, %sSize: %s%s, VIP: %s, Trusted: %s" % (index, ' ' if index < 10 else '',color["GREEN"], torrent.seeds,color["END"], torrent.title, color["PURPLE"], torrent.filesize,color["END"] , torrent.is_vip, torrent.is_trusted))
        print("[%s%s]" % ('0' if index < 10 else '', index), end='')
        #print(color["DARKCYAN"],"",end='')
        print("%s%s" % ( color["GREEN"] if torrent.is_vip else color["PURPLE"] if torrent.is_trusted else color["RED"],'🕱'),end='')
        #print(color["DARKCYAN"],"",end='')
        print(color["UNDERLINE"],end='')
        print(color["GREEN"],end='')
        print("[Seed:%s]" % (torrent.seeds), end='')
        print(color["DARKCYAN"],end='')
        print("%s " % (torrent.title), end='')
        print(color["CYAN"],end='')
        print("[Size:%s]" % (torrent.filesize),end='')
        print(color["END"])

def searchNew():
    bannerTorrentSearcher()
    keyword = ""

    while True:
        try:
            keyword = input("Search Torrent: ")
        except EOFError:
            print("%sError%s: You must put a value in order to search!" % (color["RED"],color["END"]))
            continue

        return keyword



def getChooseInput(maxRange):

    userInput = input("Choose: ")

    
    #[0]quit, [-1]error, [-2]next, [-3]back, [4]newSearch
    if userInput.isalpha():
        if userInput.lower() == 's' or userInput.lower() == "search": #newSearch
            return -4
        elif userInput.lower() == 'x' or userInput.lower() == "exit": #quit
            return 0
        elif userInput.lower() == 'n' or userInput.lower() == "next": #next
            return -2
        elif userInput.lower() == 'b' or userInput.lower() == "back": #back
            return -3
        else:
            print("\n%s Invalid Command%s" % (color["RED"],color["END"]))
            input("Press Enter to continue...")
            return -1
    try:
        userInput = int(userInput)
    except ValueError:
        print("%sError%s: You must input a value" %(color["RED"],color["END"]))
        input("Press Enter to continue...")
        return -1

    if userInput <= 0 or userInput > maxRange:
        print("%sError%s: Invalid Input: Must be between 0-%d" %(color["RED"],color["END"],maxRange))
        input("Press Enter to continue...")
        return -1

    return userInput



def detailedItemView(torrent):
    print("\nSelected torrent:\n")
    print("Name: " + torrent.title)
    print("Category: " + torrent.category)
    print("Size: " + torrent.filesize)
    print("Seeder: %d\n" % (torrent.seeds))
    print("Date Uploaded: " + torrent.upload_date)
    print("Uploader: " + torrent.uploader)
    print("Reputation: %s" %("VIP" if torrent.is_vip else "Trusted" if torrent.is_trusted else "Unverified"))
    #print(CATEGORIES.printOptions())
    #print(ORDERS.printOptions())
    print("URL: " + torrent.url)

def askToDownload(magnetlink):
    option = input("\nDownload File? Y/N: ")
    while option.lower() not in ("y","n"):
        option = input("%s Invalid Syntax%s, Download File? Y/N: " % (color["RED"],color["END"]))
    if option == "y":
        if magnet2Torrent(magnetlink,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnetlink, params))
            return True
    input("Press Enter to continue...")
    return False

def torrentSearcher():
    Search = searchNew()
    t = TPB()
    page = 1

    while True:
        print("Please Wait...")
        torrents = t.search(Search,page,ORDERS.SEEDERS.DES)
        clear_output()
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-")
        print("%sSearch Results of: %s" % (color["BOLD"],Search))
        print("Page: %s\t Current Items: %d\t Category: All\t Sort: Seeders/Descending %s" % (page,len(torrents), color["END"]))
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-")
        printContents(torrents)
        print("\n%sOptions: [S]earch, [N]ext, [B]ack, E[x]it:%s\n" % (color["BOLD"],color["END"]))
        userInput = getChooseInput(len(torrents))
        #[0]quit, [-1]error, [-2]next, [-3]back, [-4]newSearch
        
        if userInput == 0:
            print("Now Exiting Search Engine...")
            break
        
        elif userInput == -2:
            if len(torrents) == 30:
                page = page + 1
            else:
                print("Error: This is the last page.")
                input("Press Enter to continue...")
            continue

        elif userInput == -3:
            if page - 1 > 0:
                page = page - 1
                continue
            else:
                print("Error, you're in the starting page already. ")
                input("Press Enter to continue...")
                continue
        elif userInput == -4:
            Search = searchNew()
            page = 1
            continue
        elif userInput > 0:
            torrent = torrents[userInput-1]
            detailedItemView(torrent)
            if askToDownload(torrent.magnetlink):
                print ("\nImport Complete") 
                showQueue()
                input("Press Enter to continue...")
                continue
        else :
            print("%sError%s, Adding unsuccessful" % (color["RED"],color["END"]))
            input("Press Enter to continue...")





#Main
params = {"save_path": SaveDirectory}
Menu = "Search Torrents" #@param ["Search Torrents","Add Magnet", "Add torrent", "Resume Downloads"]


if Menu == "Add Magnet" :
    addViaMagnet()
elif Menu == "Add .torrent" :
    try: 
        addViaTorrent()
    except:
      print ("Selection Canceled..")
elif Menu == "Resume Downloads" :
    resumeTorrent()
    showQueue()
elif Menu == "Search Torrents" :
    torrentSearcher()


#elif Add == "resume downloads" planned feature



time.sleep(2)
clear_output()
#print (color["UNDERLINE"],color["BOLD"],"Now Downloading:",color["END"])
bannerTorrentDownloader()
state_str = [
    "Queued:",
    "🔎 Checking:",
    "\u267B Downloading Metadata:",
    "🡇 Downloading:",
    "Finished:",
    "Seeding",
    "\u221E Allocating:",
    "Checking fastresume:",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    state_str[s.state],
                    download.name(),
                    ",🌱Seeders: ",str(s.num_seeds),
                    ",⇅Speed: ", str(s.download_rate / 1000),"kB/s",
                    
                ]
            )
            bar.value = s.progress * 100  
        else:
            next_shift -= 1
            transferTorrent(download.name())
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() 
            download_bars.remove(bar)
            print("%s \u272F%s %s %sComplete%s" % (color["YELLOW"],color["END"],download.name(), color["GREEN"],color["END"]))
    time.sleep(1)

print("\n\u2705Program Terminated, No more pending downloads!")


In [ ]:
print("🕱")

🕱


In [ ]:

def bannerTorrentSearcher():
    print("%s     _____ %s                         _   %s  _____                     _       " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s    |_   _|%s                        | |  %s /  ___|                   | |      " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | |%s  ___  _ __ _ __ ___ _ __ | |_ %s \ `--.  ___  __ _ _ __ ___| |__    " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | %s|/ _ \| '__| '__/ _ | '_ \| __|%s  `--. \/ _ \/ _` | '%s__/ __| '_ \ " % (color["PURPLE"],color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | (_) | |  | | |  __| | | | |_ %s /\__/ |  __| (_| | |%s | (__| | | |" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      \_/\___/|_|  |_|  \___|_| |_|\__|%s \____/ \___|\__,_|_|%s  \___|_| |_|" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s                                                                             " % (color["END"]))
    
def printContents(torrents):
    index=0
    for torrent in torrents:
        index = index + 1
        #print("[%d]%s , %sSeeds: %s %-5s%s, %sSize: %s%s, VIP: %s, Trusted: %s" % (index, ' ' if index < 10 else '',color["GREEN"], torrent.seeds,color["END"], torrent.title, color["PURPLE"], torrent.filesize,color["END"] , torrent.is_vip, torrent.is_trusted))
        print("[%s%s]" % ('0' if index < 10 else '', index), end='')
        #print(color["DARKCYAN"],"",end='')
        print("%s%s" % ( color["GREEN"] if torrent.is_vip else color["PURPLE"] if torrent.is_trusted else color["RED"],'🕱'),end='')
        #print(color["DARKCYAN"],"",end='')
        print(color["UNDERLINE"],end='')
        print(color["GREEN"],end='')
        print("[Seed:%s]" % (torrent.seeds), end='')
        print(color["DARKCYAN"],end='')
        print("%s " % (torrent.title), end='')
        print(color["CYAN"],end='')
        print("[Size:%s]" % (torrent.filesize),end='')
        print(color["END"])

def searchNew():
    bannerTorrentSearcher()
    keyword = ""

    while True:
        try:
            keyword = input("Search Torrent: ")
        except EOFError:
            print("%sError%s: You must put a value in order to search!" % (color["RED"],color["END"]))
            continue

        return keyword



def getChooseInput(maxRange):

    userInput = input("Choose: ")

    
    #[0]quit, [-1]error, [-2]next, [-3]back, [4]newSearch
    if userInput.isalpha():
        if userInput.lower() == 's' or userInput.lower() == "search": #newSearch
            return -4
        elif userInput.lower() == 'x' or userInput.lower() == "exit": #quit
            return 0
        elif userInput.lower() == 'n' or userInput.lower() == "next": #next
            return -2
        elif userInput.lower() == 'b' or userInput.lower() == "back": #back
            return -3
        else:
            print("\n%s Invalid Command%s" % (color["RED"],color["END"]))
            input("Press Enter to continue...")
            return -1
    try:
        userInput = int(userInput)
    except ValueError:
        print("%sError%s: You must input a value" %(color["RED"],color["END"]))
        input("Press Enter to continue...")
        return -1

    if userInput <= 0 or userInput > maxRange:
        print("%sError%s: Invalid Input: Must be between 0-%d" %(color["RED"],color["END"],maxRange))
        input("Press Enter to continue...")
        return -1

    return userInput



def detailedItemView(torrent):
    print("\nSelected torrent:\n")
    print("Name: " + torrent.title)
    print("Category: " + torrent.category)
    print("Size: " + torrent.filesize)
    print("Seeder: %d\n" % (torrent.seeds))
    print("Date Uploaded: " + torrent.upload_date)
    print("Uploader: " + torrent.uploader)
    print("Reputation: %s" %("VIP" if torrent.is_vip else "Trusted" if torrent.is_trusted else "Unverified"))
    #print(CATEGORIES.printOptions())
    #print(ORDERS.printOptions())
    print("URL: " + torrent.url)

def askToDownload(magnetlink):
    option = input("\nDownload File? Y/N: ")
    while option.lower() not in ("y","n"):
        option = input("%s Invalid Syntax%s, Download File? Y/N: " % (color["RED"],color["END"]))
    if option == "y":
        if magnet2Torrent(magnetlink,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnetlink, params))
            return True
    input("Press Enter to continue...")
    return False

def torrentSearcher():
    Search = searchNew()
    t = TPB()
    page = 1
    while True:
        print("Please Wait...")
        #torrents = t.search(Search,page,ORDERS.SEEDERS.DES)
        category = dict()
        category["Games"]= CATEGORIES.GAMES.ALL
        category["Videos"]= CATEGORIES.VIDEO.ALL
       
        torrents = t.top(category["Games"])

        clear_output()
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        print("\t%sSearch Results of: %s" % (color["BOLD"],Search))
        print("\tPage: %s\t Current Items: %d\t Category: All\t Sort: Seeders/Descending %s" % (page,len(torrents), color["END"]))
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        printContents(torrents)
        print("\n%sOptions: [S]earch, [N]ext, [B]ack, E[x]it:%s\n" % (color["BOLD"],color["END"]))
        userInput = getChooseInput(len(torrents))
        #[0]quit, [-1]error, [-2]next, [-3]back, [-4]newSearch
        
        if userInput == 0:
            print("Now Exiting Search Engine...")
            break
        
        elif userInput == -2:
            if len(torrents) == 30:
                page = page + 1
            else:
                print("Error: This is the last page.")
                input("Press Enter to continue...")
            continue

        elif userInput == -3:
            if page - 1 > 0:
                page = page - 1
                continue
            else:
                print("Error, you're in the starting page already. ")
                input("Press Enter to continue...")
                continue
        elif userInput == -4:
            Search = searchNew()
            page = 1
            continue
        elif userInput > 0:
            torrent = torrents[userInput-1]
            detailedItemView(torrent)
            if askToDownload(torrent.magnetlink):
                print ("\nImport Complete") 
                showQueue()
                input("Press Enter to continue...")
                continue
        else :
            print("%sError%s, Adding unsuccessful" % (color["RED"],color["END"]))
            input("Press Enter to continue...")


torrentSearcher()


-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
	Search Results of: j
	Page: 1	 Current Items: 100	 Category: All	 Sort: Seeders/Descending 
-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
[01]🕱[Seed:594]GTA V - Grand Theft Auto V - Rjaa [Size:59.03 GiB]
[02]🕱[Seed:541]Grand Theft Auto San Andreas-HOODLUM [Size:3.94 GiB]
[03]🕱[Seed:335]Need for Speed Underground 2 V1.2 {Full-Game} {Requested ISO} {b [Size:1.12 GiB]
[04]🕱[Seed:315]Age of Empires III - Complete Collection (ENG|RUS) [Repack] R.G. [Size:2.32 GiB]
[05]🕱[Seed:315]Resident Evil 4 PC Game [Size:2.5 GiB]
[06]🕱[Seed:281]Minecraft 1.16 [Full, Online, Server List] [Size:349.23 MiB]
[07]🕱[Seed:220]Mini Motorways [Size:125.11 MiB]
[08]🕱[Seed:216]Garrys Mod v2020.03.17 Full +AutoUpdate +Multilanguage [Size:1.69 GiB]
[09]🕱[Seed:214]Stardew Valley v1.5.4.981587505 (GOG) [Size:546.75 MiB]
[10]🕱[Seed:213]Age of Empires II: Definitive Edition - CODEX

KeyboardInterrupt: ignored

In [ ]:
print(CATEGORIES.printOptions())

ALL
AUDIO.ALL
AUDIO.MUSIC
AUDIO.AUDIO_BOOKS
AUDIO.SOUND_CLIPS
AUDIO.FLAC
AUDIO.OTHER
VIDEO.ALL
VIDEO.MOVIES
VIDEO.MOVIES_DVDR
VIDEO.MUSIC_VIDEOS
VIDEO.MOVIE_CLIPS
VIDEO.TV_SHOWS
VIDEO.HANDHELD
VIDEO.HD_MOVIES
VIDEO.HD_TV_SHOWS
VIDEO.THREE_DIMENSIONS
VIDEO.OTHER
APPLICATIONS.ALL
APPLICATIONS.WINDOWS
APPLICATIONS.MAC
APPLICATIONS.UNIX
APPLICATIONS.HANDHELD
APPLICATIONS.IOS
APPLICATIONS.ANDROID
APPLICATIONS.OTHER
GAMES.ALL
GAMES.PC
GAMES.MAC
GAMES.PSX
GAMES.XBOX360
GAMES.WII
GAMES.HANDHELD
GAMES.IOS
GAMES.ANDROID
GAMES.OTHER
PORN.ALL
PORN.MOVIES
PORN.MOVIES_DVDR
PORN.PICTURES
PORN.GAMES
PORN.HD_MOVIES
PORN.MOVIE_CLIPS
PORN.OTHER
OTHER.EBOOKS
OTHER.COMICS
OTHER.PICTURES
OTHER.COVERS
OTHER.PHYSIBLES
OTHER.OTHER
None


In [ ]:
print(ORDERS.printOptions())

NAME.DES
NAME.ASC
UPLOADED.DES
UPLOADED.ASC
SIZE.DES
SIZE.ASC
SEEDERS.DES
SEEDERS.ASC
LEECHERS.DES
LEECHERS.ASC
UPLOADER.DES
UPLOADER.ASC
TYPE.DES
TYPE.ASC
None
